# San Fransisco Housing Market Analysis

In [48]:
# importing libraries
import pandas as pd
import hvplot.pandas
from pathlib import Path
import warnings
import geoviews as gv
import holoviews as hv

In [49]:
warnings.filterwarnings('ignore')

In [50]:
# reading in the census data
sfo_data_df = pd.read_csv('./Resources/sfo_neighborhoods_census_data.csv', index_col='year')

# displaying the dataframe
display(sfo_data_df.head())
display(sfo_data_df.tail())

,neighborhood,sale_price_sqr_foot,housing_units,gross_rent
year,,,,
2010,Alamo Square,291.182945,372560,1239
2010,Anza Vista,267.932583,372560,1239
2010,Bayview,170.098665,372560,1239
2010,Buena Vista Park,347.394919,372560,1239
2010,Central Richmond,319.027623,372560,1239


,neighborhood,sale_price_sqr_foot,housing_units,gross_rent
year,,,,
2016,Telegraph Hill,903.049771,384242,4390
2016,Twin Peaks,970.085470,384242,4390
2016,Van Ness/ Civic Center,552.602567,384242,4390
2016,Visitacion Valley,328.319007,384242,4390
2016,Westwood Park,631.195426,384242,4390


---

In [51]:
# grouping the dataframe by year and dropping neighborhood so you can see San Fransisco's data as a whole
housing_units = sfo_data_df.drop(columns="neighborhood")
housing_units_by_year = housing_units.groupby("year").mean()

housing_units_by_year

,sale_price_sqr_foot,housing_units,gross_rent
year,,,
2010,369.344353,372560.0,1239.0
2011,341.903429,374507.0,1530.0
2012,399.389968,376454.0,2324.0
2013,483.600304,378401.0,2971.0
2014,556.277273,380348.0,3528.0
2015,632.540352,382295.0,3739.0
2016,697.643709,384242.0,4390.0


In [52]:
# plotting the housing units per year for SF
housing_units_by_year.hvplot.bar(
    title='Housing units in San Fransisco from 2010 to 2016',
    x='year',
    y='housing_units',
    xlabel='Year',
    ylabel='Housing Units',
    fill_color='blue',
    ylim=[365000, 385000],
    width=750,
    height=350,
    hover_cols=['sale_price_sqr_foot', 'gross_rent'],
    hover_color='green'
).opts(
    yformatter='%.0f'
)

:Bars   [year]   (housing_units,sale_price_sqr_foot,gross_rent)

In [53]:
# calculating percent change of the housing units per year
avg_change_housing_units = housing_units_by_year['housing_units'].pct_change().dropna().mean()

print(f"The average annual change in housing units was {round(100*avg_change_housing_units, 2)}%")

The average annual change in housing units was 0.52%


**Question:** What is the overall trend in housing_units over the period being analyzed?

**Answer:** The housing units rose at a constant rate over the 6 years.

---

In [54]:
# creating a new dataframe and grouping it by year
prices_square_foot_by_year = housing_units.groupby('year').mean()

prices_square_foot_by_year

,sale_price_sqr_foot,housing_units,gross_rent
year,,,
2010,369.344353,372560.0,1239.0
2011,341.903429,374507.0,1530.0
2012,399.389968,376454.0,2324.0
2013,483.600304,378401.0,2971.0
2014,556.277273,380348.0,3528.0
2015,632.540352,382295.0,3739.0
2016,697.643709,384242.0,4390.0


**Question:** What is the lowest gross rent reported for the years included in the DataFrame?

**Answer:** The lowest gross income was in 2010

In [55]:
# taking housing units out of the dataframe
prices_square_foot_by_year = prices_square_foot_by_year.drop(columns=['housing_units'])

prices_square_foot_by_year

,sale_price_sqr_foot,gross_rent
year,,
2010,369.344353,1239.0
2011,341.903429,1530.0
2012,399.389968,2324.0
2013,483.600304,2971.0
2014,556.277273,3528.0
2015,632.540352,3739.0
2016,697.643709,4390.0


In [56]:
# calculating percent change in sales price per square foot for each year
pct_change_sales = prices_square_foot_by_year['sale_price_sqr_foot'].pct_change().mean()*100
pct_change_sales

11.583167921308576

In [57]:
# calculating percent change in rental income for each year
pct_change_rent = prices_square_foot_by_year['gross_rent'].pct_change().mean()*100
pct_change_rent

24.22695553875985

In [58]:
# plotting sales price and rental income per year 
prices_square_foot_by_year.hvplot.line(
    x='year',
    y=['sale_price_sqr_foot', 'gross_rent'],
    title='Sales Price Per Square Foot and Gross Rent from 2010 to 2016',
    xlabel='Year',
    ylabel='Sales Price Per Square Foot & Gross Rent'
)

:NdOverlay   [Variable]
   :Curve   [year]   (value)

**Question:** Did any year experience a drop in the average sale price per square foot compared to the previous year?

**Answer:** 2011 had a slight dip in price compared to 2010

**Question:** If so, did the gross rent increase or decrease during that year?

**Answer:** The gross rent, however, increased between 2010 and 2011

---

In [59]:
# making a new dataframe grouped by year and neighborhood
prices_by_year_by_neighborhood = sfo_data_df.groupby(['year', 'neighborhood']).mean()

prices_by_year_by_neighborhood

sale_price_sqr_foot  housing_units  gross_rent
year neighborhood                                                          
2010 Alamo Square                     291.182945       372560.0      1239.0
     Anza Vista                       267.932583       372560.0      1239.0
     Bayview                          170.098665       372560.0      1239.0
     Buena Vista Park                 347.394919       372560.0      1239.0
     Central Richmond                 319.027623       372560.0      1239.0
...                                          ...            ...         ...
2016 Telegraph Hill                   903.049771       384242.0      4390.0
     Twin Peaks                       970.085470       384242.0      4390.0
     Van Ness/ Civic Center           552.602567       384242.0      4390.0
     Visitacion Valley                328.319007       384242.0      4390.0
     Westwood Park                    631.195426       384242.0      4390.0

[397 rows x 3 columns]

In [60]:
# taking housing units out of the dataframe
prices_by_year_by_neighborhood = prices_by_year_by_neighborhood.drop(columns='housing_units')

# displaying the dataframe
display(prices_by_year_by_neighborhood.head())
display(prices_by_year_by_neighborhood.tail())

sale_price_sqr_foot  gross_rent
year neighborhood                                     
2010 Alamo Square               291.182945      1239.0
     Anza Vista                 267.932583      1239.0
     Bayview                    170.098665      1239.0
     Buena Vista Park           347.394919      1239.0
     Central Richmond           319.027623      1239.0

sale_price_sqr_foot  gross_rent
year neighborhood                                           
2016 Telegraph Hill                   903.049771      4390.0
     Twin Peaks                       970.085470      4390.0
     Van Ness/ Civic Center           552.602567      4390.0
     Visitacion Valley                328.319007      4390.0
     Westwood Park                    631.195426      4390.0

In [61]:
# showing the sales price and rental income per year for each neighborhood
prices_by_year_by_neighborhood.hvplot.line(
    x='year',
    title='Sales Price Per Square Foot and Gross Rent by Neighborhood',
    xlabel='Year',
    ylabel='Sales Price Per Square Foot & Gross Rent',
    groupby='neighborhood'
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'ba1f57a4-ec68-4000-bbec-7498d28f2756': {'version…

**Question:** For the Anza Vista neighborhood, is the average sale price per square foot for 2016 more or less than the price that’s listed for 2012? 

**Answer:** The average sale price for 2016 is much lower than in 2012 for the Anza Vista neighborhood

---

In [62]:
# reading in the coordinates data
neighborhood_locations_df = pd.read_csv(
    Path('./Resources/neighborhoods_coordinates.csv'),
    index_col='Neighborhood'
)

neighborhood_locations_df

,Lat,Lon
Neighborhood,,
Alamo Square,37.791012,-122.402100
Anza Vista,37.779598,-122.443451
Bayview,37.734670,-122.401060
Bayview Heights,37.728740,-122.410980
Bernal Heights,37.728630,-122.443050
...,...,...
West Portal,37.740260,-122.463880
Western Addition,37.792980,-122.435790
Westwood Highlands,37.734700,-122.456854


In [63]:
# grouping the dataframe by neighborhood
all_neighborhood_info_df = sfo_data_df.groupby('neighborhood').mean()

all_neighborhood_info_df

,sale_price_sqr_foot,housing_units,gross_rent
neighborhood,,,
Alamo Square,366.020712,378401.00,2817.285714
Anza Vista,373.382198,379050.00,3031.833333
Bayview,204.588623,376454.00,2318.400000
Bayview Heights,590.792839,382295.00,3739.000000
Bernal Heights,576.746488,379374.50,3080.333333
...,...,...,...
West Portal,498.488485,376940.75,2515.500000
Western Addition,307.562201,377427.50,2555.166667
Westwood Highlands,533.703935,376454.00,2250.500000


In [64]:
# concatinating the dataframes
all_neighborhoods_df = pd.concat(
    [neighborhood_locations_df, all_neighborhood_info_df], 
    axis="columns",
    sort=False
)

# displaying the new dataframe
display(all_neighborhoods_df.head())
display(all_neighborhoods_df.tail())


,Lat,Lon,sale_price_sqr_foot,housing_units,gross_rent
Alamo Square,37.791012,-122.402100,366.020712,378401.0,2817.285714
Anza Vista,37.779598,-122.443451,373.382198,379050.0,3031.833333
Bayview,37.734670,-122.401060,204.588623,376454.0,2318.400000
Bayview Heights,37.728740,-122.410980,590.792839,382295.0,3739.000000
Bernal Heights,37.728630,-122.443050,NaN,NaN,NaN


,Lat,Lon,sale_price_sqr_foot,housing_units,gross_rent
Yerba Buena,37.79298,-122.39636,576.709848,377427.5,2555.166667
Bernal Heights,NaN,NaN,576.746488,379374.5,3080.333333
Downtown,NaN,NaN,391.434378,378401.0,2817.285714
Ingleside,NaN,NaN,367.895144,377427.5,2509.000000
Outer Richmond,NaN,NaN,473.900773,378401.0,2817.285714


In [65]:
# dropping all nulls
all_neighborhoods_df = all_neighborhoods_df.reset_index().dropna()

# renaming the index to Neighborhood and rounding all the data for sales price and rental income
all_neighborhoods_df = all_neighborhoods_df.rename(columns={"index": "Neighborhood"})
all_neighborhoods_df = all_neighborhoods_df.round({'sale_price_sqr_foot':2, 'gross_rent':2})

# Review the resulting DataFrame
display(all_neighborhoods_df.head())
display(all_neighborhoods_df.tail())

,Neighborhood,Lat,Lon,sale_price_sqr_foot,housing_units,gross_rent
0,Alamo Square,37.791012,-122.402100,366.02,378401.0,2817.29
1,Anza Vista,37.779598,-122.443451,373.38,379050.0,3031.83
2,Bayview,37.734670,-122.401060,204.59,376454.0,2318.40
3,Bayview Heights,37.728740,-122.410980,590.79,382295.0,3739.00
5,Buena Vista Park,37.768160,-122.439330,452.68,378076.5,2698.83


,Neighborhood,Lat,Lon,sale_price_sqr_foot,housing_units,gross_rent
68,West Portal,37.74026,-122.463880,498.49,376940.75,2515.50
69,Western Addition,37.79298,-122.435790,307.56,377427.50,2555.17
70,Westwood Highlands,37.73470,-122.456854,533.70,376454.00,2250.50
71,Westwood Park,37.73415,-122.457000,687.09,382295.00,3959.00
72,Yerba Buena,37.79298,-122.396360,576.71,377427.50,2555.17


In [66]:
# creating a point plot to see the neighborhoods sales price and rental income on a map
all_neighborhoods_df.hvplot.points(
    geo=True,
    x='Lon',
    y='Lat',
    size='sale_price_sqr_foot',
    color='gross_rent',
    frame_width=700,
    frame_height=500,
    tiles='OSM',
    title='Sales Price Per Square Foot and Gross Rent by Neighborhood'
)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lon,Lat]   (gross_rent,sale_price_sqr_foot)

**Question:** Which neighborhood has the highest gross rent, and which has the highest sale price per square foot?

**Answer:** Yerba Buena has the highest gross rent and the highest sale price per square foot

**Question:**  How does the trend in rental income growth compare to the trend in sales prices? Does this same trend hold true for all the neighborhoods across San Francisco?

**Answer:** Rental income outgrew sales prices in every neighborhood

**Question:** What insights can you share with your company about the potential one-click, buy-and-rent strategy that they're pursuing? Do neighborhoods exist that you would suggest for investment, and why?

**Answer:** I believe it's a good strategy if you put it all in context. For example, the Silver Terrace and Visitacion Valley neighborhoods have low sales prices, $170/sqft and $301/sqft respectively, but also have some of the highest rents, $3,528/month and $3,657/month respectively. Given this information, it would seem that if your one click was on these neighborhoods then it would make a great investment. 